In [0]:
import tensorflow as tf 
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import itertools
import collections
import operator

In [0]:
anime = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Anime Recommendation/anime.csv')
user_ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Anime Recommendation/rating.csv')

## Cleaning the Data

In [0]:
anime = anime.dropna()
user_ratings['rating'] = user_ratings['rating'].replace(-1, np.nan)
user_ratings = user_ratings.dropna()

In [0]:
len(anime[anime['type'] == 'Movie'])

2259

Filtered based on movies only and #ratings > 1000


In [0]:
movies_df = anime[ (anime['type'] == 'Movie') & (anime['members'] > 1000) ]

In [0]:
len(movies_df)

907

In [0]:
movies_users = movies_df.merge(user_ratings, 
                               left_on='anime_id', 
                               right_on='anime_id', 
                               suffixes=['_anime', '_user'])

In [0]:
len(movies_users)

836945

In [0]:
num_ratings = movies_users.groupby('user_id')['rating_user'].agg('count')

In [0]:
# We wil define reliable users as the ones who have rated more than 200 movies
rel_users = num_ratings[num_ratings >= 200]
len(rel_users)

54

In [0]:
movies_users['reliable user'] = movies_users['user_id'].map(lambda x: True if x in rel_users.index.values else False)

In [0]:
movies_users = movies_users[movies_users['reliable user'] == True]

In [0]:
len(movies_users)

13277

## 2. Collaborative Filtering based on user similarity

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
%matplotlib inline

In [0]:
# Create a data from with the user id, anime title and ratings
anime_colab = movies_users[['user_id', 'name', 'rating_user']]
anime_colab.rename(columns = {'name':'anime_title'}, inplace = True)
anime_colab.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,user_id,anime_title,rating_user
153,5669,Kimi no Na wa.,9.0
165,6569,Kimi no Na wa.,10.0
332,13954,Kimi no Na wa.,6.0
505,18579,Kimi no Na wa.,10.0
658,25252,Kimi no Na wa.,10.0


In [0]:
len(anime_colab)

13277

In [0]:
# anime_colab = anime_colab[anime_colab['user_id'] <= 10000]

In [0]:
# To find the similarity between users, we will be using the Cosine similarity. For this we will create a pivit table with
# users along one axis and movies along the other
anime_piv = anime_colab.pivot_table(index='user_id', columns='anime_title', values='rating_user')

In [0]:
anime_piv.shape

(54, 866)

In [0]:
anime_piv

anime_title,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Trilogy,.hack//The Movie: Sekai no Mukou ni,009 Re:Cyborg,00:08,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,663114,Aa! Megami-sama! Movie,Aachi wa Ssipak,Accel World: Infinite∞Burst,Acchi Kocchi,Ace wo Nerae! (1979),Afro Samurai Movie,Afro Samurai: Resurrection,Ai,Ai City,Aikatsu! Movie,Aikatsu! Music Award: Minna de Shou wo MoracchaimaShow!,Air Movie,Ajin Part 1: Shoudou,Ajin Part 2: Shoutotsu,Akage no Anne: Green Gables e no Michi,Aki no Kanade,Akihabara Dennou-gumi: 2011-nen no Natsuyasumi,Akira,Alexander Senki Movie,Alps no Shoujo Heidi (1979),Amanatsu,Andersen Douwa: Ningyohime,Anime Tenchou Movie,Anne no Nikki,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai. Movie,Ansatsu Kyoushitsu: 365-nichi no Jikan,Ao no Exorcist Movie,Aoki Densetsu Shoot! (Movie),Aoki Hagane no Arpeggio: Ars Nova Cadenza,Aoki Hagane no Arpeggio: Ars Nova DC,Aos,...,Urusei Yatsura Movie 1: Only You,Urusei Yatsura Movie 2: Beautiful Dreamer,Urusei Yatsura Movie 3: Remember My Love,Urusei Yatsura Movie 4: Lum The Forever,Urusei Yatsura Movie 5: Final,Urusei Yatsura Movie 6: Itsudatte My Darling,Ushiro no Shoumen Daare,Vampire Hunter D,Vampire Hunter D (2000),Venus Senki,Vexille: 2077 Nihon Sakoku,Waga Seishun no Arcadia,"Wake Up, Girls! Beyond the Bottom","Wake Up, Girls! Seishun no Kage","Wake Up, Girls! Shichinin no Idol",Wasurenagumo,Wata no Kuni Hoshi,Windaria,Wonderful Days,X/1999,Xi Avant,Xiao Qian,Yamato yo Towa ni,Yes! Precure 5 GoGo! Movie: Okashi no Kuni no Happy Birthday,Yes! Precure 5 Movie: Kagami no Kuni no Miracle Daibouken!,Yodaka no Hoshi,Youjuu Toshi,Youkai Watch Movie 1: Tanjou no Himitsu da Nyan!,Youkoso Lodoss-tou e!,Yowamushi Pedal Movie,Yowamushi Pedal: Re:RIDE,Yowamushi Pedal: Re:ROAD,Yowamushi Pedal: Spare Bike,Yuu☆Yuu☆Hakusho (Movie),Yuu☆Yuu☆Hakusho: Meikai Shitou-hen - Honoo no Kizuna,Yu☆Gi☆Oh! (1999),Yu☆Gi☆Oh!: Hikari no Pyramid,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,eX-Driver the Movie,xxxHOLiC Movie: Manatsu no Yoru no Yume
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1530,8.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,7.0,7.0,NaN,7.0,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,6.0,6.0,7.0,NaN,NaN,NaN
5584,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
5669,7.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,7.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,8.0
5899,8.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,7.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6569,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,8.0,NaN,NaN,8.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,6.0,6.0,NaN,NaN,NaN,7.0,7.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,7.0,7.0,NaN,7.0,NaN,NaN,NaN,7.0,NaN,NaN
7114,10.0,7.0,8.0,NaN,4.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,10.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,10.0,9.0,NaN,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,8.0

In [0]:
# Normalize the ratings
df = anime_piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

In [0]:
df.shape

(54, 866)

In [0]:
# Replacing NO user ratings with value of 0
df.fillna(0, inplace=True)
df = df.T

# Removing the users who did not rate any movies
df = df.loc[:, (df != 0).any(axis=0)]

In [0]:
df.shape

(866, 54)

In [0]:
df.shape

# Converting to a sparse matrix and computing cosine similarity
df_sparse = sp.sparse.csr_matrix(df.values)
anime_similarity = cosine_similarity(df_sparse)
user_similarity = cosine_similarity(df_sparse.T)

# Converting similarity matrices to data frames
anime_sim_df = pd.DataFrame(anime_similarity, index = df.index, columns = df.index)
user_sim_df = pd.DataFrame(user_similarity, index = df.columns, columns = df.columns)

In [0]:
user_sim_df.head()

user_id,1530,5584,5669,5899,6569,7114,7340,7345,7511,8115,9032,11359,11536,13954,18579,22434,22910,24403,25252,25354,27219,32687,39921,40273,41462,42635,45659,47686,48373,49192,49503,49662,50995,51270,51693,51806,52175,52776,53698,53700,53713,54069,56619,57620,58438,59245,59406,59643,65836,65840,66742,67708,68797,71792
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1530,1.000000,0.015355,-0.098620,0.013471,0.013347,0.008078,0.015283,0.060739,0.031187,0.032109,0.007275,-0.002178,0.119528,0.039517,0.087114,0.170103,-0.058923,-0.002566,0.060710,0.074594,0.064300,0.030276,0.117989,0.024472,0.075794,0.037231,0.047074,-0.044924,-0.037313,-0.059895,0.059741,0.034886,0.032467,0.046080,-0.010910,-0.094090,0.052210,0.086290,0.013205,0.082309,-0.054476,0.067002,0.120922,0.014288,0.008589,0.007572,-0.006373,0.060640,-0.033460,0.050461,0.027714,-0.012923,0.007426,0.067517
5584,0.015355,1.000000,0.298264,0.338483,0.154229,0.045226,0.270552,0.282819,0.060514,0.197485,0.089385,0.114623,0.073979,-0.042541,-0.138380,0.215171,0.151829,0.124723,-0.037642,0.289299,0.038241,0.157110,0.188906,0.162887,0.091956,0.156337,0.184334,0.060484,0.194257,0.210436,0.197216,0.136691,0.139833,0.159665,0.107889,0.096133,0.192563,0.108970,0.089511,0.280008,0.133236,0.229503,-0.111715,0.206209,0.274622,0.246241,0.138338,0.258618,0.037903,0.162477,0.064679,0.228375,0.304899,0.107895
5669,-0.098620,0.298264,1.000000,0.248043,0.261406,0.149189,0.146523,0.299704,0.103847,0.341229,0.139051,0.142580,0.223559,0.126284,-0.096579,0.195378,0.215531,0.203691,0.117556,0.325796,0.143305,0.078007,0.265499,0.199933,0.183292,0.209869,0.183687,0.104711,0.276465,0.246504,0.291794,0.236834,0.238545,0.221472,0.247944,0.184205,0.231342,0.246515,0.139970,0.253622,0.024082,0.251325,-0.035759,0.255202,0.302543,0.242708,0.224819,0.321120,0.074759,0.220219,0.226930,0.256555,0.388000,0.219536
5899,0.013471,0.338483,0.248043,1.000000,0.220949,0.060861,0.113563,0.283470,0.042437,0.232571,0.257387,0.162387,0.216880,0.044067,-0.141662,0.207319,0.094624,0.201207,0.079159,0.298316,0.067646,0.195214,0.229344,0.167464,0.176261,0.108697,0.250696,0.070521,0.180389,0.187432,0.299056,0.168265,0.140789,0.294613,0.192371,0.160362,0.296236,0.161240,0.184291,0.296258,0.059427,0.197973,-0.012365,0.234655,0.198965,0.242465,0.200095,0.332053,-0.015059,0.196032,0.160034,0.268648,0.323020,0.183894
6569,0.013347,0.154229,0.261406,0.220949,1.000000,0.158657,0.172812,0.220377,0.083186,0.236628,0.182712,0.139186,0.199170,0.071752,-0.011881,0.322545,0.079403,0.187997,0.039879,0.231666,0.125558,0.087148,0.223684,0.128060,0.282388,0.148803,0.158464,0.130487,0.134486,0.233804,0.193266,0.239821,0.091950,0.291509,0.309913,0.174741,0.162553,0.106502,0.211456,0.160357,-0.006750,0.104285,0.092100,0.178634,0.132958,0.218170,0.173723,0.338904,0.054500,0.207008,0.135182,0.181004,0.191595,0.049089


In [0]:
# Function will return the top 'n' similar users for aa particular user
# 'n' can be given as a parmater here

def get_topUsers(user_id, n):
  if user_id not in df.columns:
        return('No data available on user {}'.format(user_id))

  sim_values = user_sim_df[user_id].sort_values(ascending=False)[1:n+1]
  sim_users = user_sim_df[user_id].sort_values(ascending=False)[1:n+1].index.values
  iterator = zip(sim_users, sim_values)
  print('{} Similar Users to user #{}: \n'.format(n, user_id))
  for user, values in iterator:
    print('User #{0}, Similarity value: {1:.2f}'.format(user, values))

In [0]:
get_topUsers(5899, 15)

15 Similar Users to user #5899: 

User #5584, Similarity value: 0.34
User #59643, Similarity value: 0.33
User #68797, Similarity value: 0.32
User #49503, Similarity value: 0.30
User #25354, Similarity value: 0.30
User #53700, Similarity value: 0.30
User #52175, Similarity value: 0.30
User #51270, Similarity value: 0.29
User #7345, Similarity value: 0.28
User #67708, Similarity value: 0.27
User #9032, Similarity value: 0.26
User #45659, Similarity value: 0.25
User #5669, Similarity value: 0.25
User #59245, Similarity value: 0.24
User #57620, Similarity value: 0.23


In [0]:
# Function that will return the top recommendations for a partiular user
# This takes into account the similarity between other users

def get_Recomm(user_id, num_recomm):
  if user_id not in df.columns:
    return "No data for user #{}".format(user_id)
  
  # Assessing the ratings for the top 5 similar user
  sim_users = user_sim_df[user_id].sort_values(ascending=False)[1:6].index.values
  top_animes = []
  anime_freq = {}

  for user in sim_users:
    max_score = df[user].max()
    top_animes.append(df[df.loc[:, user] == max_score].index.tolist())
  
  for anime_list in top_animes:
    for anime in anime_list:
      if anime in anime_freq:
        anime_freq[anime] += 1
      else:
        anime_freq[anime] = 1
  print('Top {} anime recommendations for user #{}\n'.format(num_recomm, user_id))
  return sorted(anime_freq.items(), key=operator.itemgetter(1), reverse=True)[:num_recomm]


In [0]:
get_Recomm(5899, 7)

Top 7 anime recommendations for user #5899



[('Sen to Chihiro no Kamikakushi', 5),
 ('Mononoke Hime', 4),
 ('Byousoku 5 Centimeter', 3),
 ('Kara no Kyoukai 5: Mujun Rasen', 3),
 ('Kara no Kyoukai 7: Satsujin Kousatsu (Kou)', 3),
 ('Kara no Kyoukai 3: Tsuukaku Zanryuu', 2),
 ('Tengen Toppa Gurren Lagann Movie: Lagann-hen', 2)]

In [0]:
# Calculate the predicted rating for a particular user for an anime
# We calculate this as a weighted average of ratings by similar users with their similarity value to the given user

def predict_rating(user_id, anime_title):
  
  if user_id not in df.columns:
    return "No data for user #{}".format(user_id)

  # Calculating with respect to the top 1000 similar users 
  sim_users = user_sim_df[user_id].sort_values(ascending=False).index[1:10]   
  sim_values = user_sim_df[user_id].sort_values(ascending=False)[1:10].tolist()

  ratings = []
  weights = []
  for index, user in enumerate(sim_users):
    rating = anime_piv.loc[user, anime_title]
    sim = sim_values[index]
    if np.isnan(rating):
      continue
    else:
      ratings.append(rating*sim)
      weights.append(sim)

  print("Predicted rating for user #{} for anime {} is: ".format(user_id, anime_title))
  return round( sum(ratings)/sum(weights), 2)

In [0]:
predict_rating(39921, 'Ai')

Predicted rating for user #39921 for anime Ai is: 


5.43

In [0]:
anime_piv.loc[39921, 'Yu☆Gi☆Oh!: Hikari no Pyramid']

4.0

In [0]:
predict_rating(39921, 'Hotaru no Haka')

Predicted rating for user #39921 for anime Hotaru no Haka is: 


9.26

In [0]:
u = 39921
a = 'Hotaru no Haka'
print('Actual rating of user {} for anime {} is: \n{}'.format(u, a, anime_piv.loc[u,a]))

Actual rating of user 39921 for anime Hotaru no Haka is: 
9.0


In [0]:
predict_rating(39921, 'Tsumiki no Ie')

Predicted rating for user #39921 for anime Tsumiki no Ie is: 


8.73

In [0]:
predict_rating(39921, 'One Piece Film: Strong World')

Predicted rating for user #39921 for anime One Piece Film: Strong World is: 


8.413329734210393

In [0]:
predict_rating(39921, 'Dareka no Manazashi')

Predicted rating for user #39921 for anime Dareka no Manazashi is: 


7.48

In [0]:
u = 39921
a = 'Dareka no Manazashi'
print('Actual rating of user {} for anime {} is: \n{}'.format(u, a, anime_piv.loc[u,a]))

Actual rating of user 39921 for anime Dareka no Manazashi is: 
7.0


In [0]:
predict_rating(39921, 'Aos')

Predicted rating for user #39921 for anime Aos is: 


5.0

In [0]:
u = 39921
a = 'Aos'
print('Actual rating of user {} for anime {} is: \n{}'.format(u, a, anime_piv.loc[u,a]))

Actual rating of user 39921 for anime Aos is: 
2.0
